### Bhagwad Gita Q&A project

In [19]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer,util
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import pandas as pd
import re
import os
import string
import pickle

In [3]:
embeded_par = pickle.load(open('models/embeded_par.pkl','rb'))
embedding_model = pickle.load(open('models/embeded_model.pkl','rb'))
clean_text_list = pickle.load(open('models/clean_text.pkl','rb'))


In [17]:
question = 'Th rough constant effort over many lifetimes'

In [18]:
embed_question = embedding_model.encode(question)

In [20]:
util.cos_sim(embeded_par,embed_question)

C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\util.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  a = torch.tensor(a)


tensor([[ 0.1027],
        [-0.0039],
        [ 0.1351],
        ...,
        [ 0.1289],
        [ 0.2097],
        [ 0.1289]])

In [26]:
util.semantic_search(embeded_par[2],embed_question)[0][0]['score']

0.13506686687469482

In [36]:
search_score = []
for i in range(len(embeded_par)):
    score = util.semantic_search(embeded_par[i],embed_question)[0][0]['score']
    search_score.append(score)
answer = clean_text_list[pd.DataFrame({'simlarity_score':search_score}).sort_values(['simlarity_score'] , ascending=False).index[0]]
answer

'  \n  \n   Th  rough constant eff  ort over many lifetimes, \na person becomes purifi  ed of all selfi  sh desires \nand attains the supreme goal of life'

In [35]:
clean_text_list[pd.DataFrame({'simlarity_score':search_score}).sort_values(['simlarity_score'] , ascending=False).index[0]]

'  \n  \n   Th  rough constant eff  ort over many lifetimes, \na person becomes purifi  ed of all selfi  sh desires \nand attains the supreme goal of life'

In [15]:
clean_text_list[2]

' And here we’re south, in the full snows of winter'

In [16]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

In [42]:
# QAmodel = SentenceTransformer('clips/mfaq')

# download size  --> 1.11 GB

In [3]:
reader = PdfReader(r"C:\Users\Ranjit Singh\Desktop\working project\bhagwadGita\bhagwad.pdf")
raw_text = ''
temp = 7
for i, page in enumerate(reader.pages):
    if i > temp:
        text = page.extract_text()
        if text:
            raw_text += text
            

In [4]:
print(raw_text)

 Th e Bhagavad Gita ╯
9 ╯Th rown together like this, they form a kind of ecstatic slide- 
show – snapshots of towering peaks of consciousness taken at 
various times by diff  erent observers and dispatched with just the barest kind of explanation. But those who have traveled those heights will recognize the views: “Oh, yes, that’s Ever-est from the northwest – must be late spring. And here we’re south, in the full snows of winter. ”
Th e Dhammapada, too, is a collection – traditionally, say-
ings of the Buddha, one of the very greatest of these explorers of consciousness. In this case the messages have been sorted, but not by a scheme that makes sense to us today. Instead of being grouped by theme or topic, they are gathered according to some dominant characteristic like a symbol or metaphor – fl owers, birds, a river, the sky – that makes them easy to com-
mit to memory. If the Upanishads are like slides, the Dham-mapada seems more like a fi  eld guide. Th  is is lore picked up 
by so

In [4]:
ls_sent = re.compile('[.!]').split(raw_text)

In [5]:
len(ls_sent)

3011

In [6]:
ls_sent[8]

' He can’t take us there, he explains, but he can show us the way: tell us what to look for, warn about missteps, advise us about detours, tell us what to avoid'

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
re.sub('[^a-zA-Z0-9`]'," ",ls_sent[8])

' He can t take us there  he explains  but he can show us the way  tell us what to look for  warn about missteps  advise us about detours  tell us what to avoid'

In [18]:
re.sub('[0-9:{|}~@[\\]/#$%&\'()*+]'," ",ls_sent[8])

' He can’t take us there, he explains, but he can show us the way  tell us what to look for, warn about missteps, advise us about detours, tell us what to avoid'

In [27]:
lem = WordNetLemmatizer()
# tokens = word_tokenize(re.sub('[^a-zA-Z0-9]'," ",ls_sent[25]))
tokens = re.sub('[0-9:{|}~@[\\]/#$%&\'()*+╯╭]'," ",ls_sent[8])
for token in tokens:
    print(f"{token}   >>> {lem.lemmatize(token)}")
    
## to perform lemmatization   \n  

In [7]:
clean_text = []
for sent in ls_sent:
    clean_text.append(re.sub('[0-9:{|}~@[\\]/#$%&\'()*+╯╭]'," ",sent))

In [8]:
len(clean_text)

3011

In [9]:
import pickle
pickle.dump(clean_text,open('clean_text.pkl','wb'))

In [10]:
print(clean_text[25])

 It is for such kindred spirits that these texts were originally composed, and it is for them in our own time that    foreword
      Th e Bhagavad Gita  
    I undertook these translations, in the conviction that they 
deserve an audience today as much as ever


#### sentence embedding

In [11]:
embeddings = []
for i in range(len(clean_text)):
    embeddings.append(embedding_model.encode(clean_text[i]))
    
## embedding the sentences

In [19]:
os.makedirs('models',exist_ok=True)
pickle.dump(embeddings,open('models/embeded_par.pkl','wb'))
pickle.dump(embedding_model,open('models/embeded_model.pkl','wb'))
pickle.dump(clean_text,open('models/clean_text.pkl','wb'))



In [17]:
# embeddings[2]
# clean_text[2]

' And here we’re south, in the full snows of winter'

In [32]:
## testing the simlarity score on the embeddigns
question = "Th rough constant effort over many lifetimes"
embed_question  = embedding_model.encode(question)

#### similarity score on text documents

In [33]:
sim_score  = util.cos_sim(embeddings,embed_question)

C:\Users\Ranjit Singh\AppData\Local\Programs\Python\Python310\lib\site-packages\sentence_transformers\util.py:36: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  a = torch.tensor(a)


In [34]:
sim_score.shape

torch.Size([3011, 1])

In [35]:
len(embeddings)

3011

In [36]:
sim_score.shape

torch.Size([3011, 1])

In [37]:
type(sim_score)

torch.Tensor

In [38]:
df = pd.DataFrame(sim_score,columns=['simlarity'])
df.head()


,simlarity
0,0.102651
1,-0.003892
2,0.135067
3,0.068451
4,0.033353


In [47]:
df.sort_values(['simlarity'],ascending=False)

,simlarity
1419,0.540046
1015,0.414484
1359,0.391690
668,0.365987
900,0.364287
...,...
86,-0.089994
2710,-0.102984
2656,-0.105293
196,-0.114648


In [39]:

df.sort_values(['simlarity'],ascending=False).index[0]

1419

In [40]:
question

## it was my question

'Th rough constant effort over many lifetimes'

In [41]:
# output return based on the similarity score
clean_text[df.sort_values(['simlarity'],ascending=False).index[0]].strip()

'Th  rough constant eff  ort over many lifetimes, \na person becomes purifi  ed of all selfi  sh desires \nand attains the supreme goal of life'

In [42]:
# output return based on the similarity score
clean_text[df.sort_values(['simlarity'],ascending=False).index[0]].strip()

'Th  rough constant eff  ort over many lifetimes, \na person becomes purifi  ed of all selfi  sh desires \nand attains the supreme goal of life'

In [43]:
clean_text[df.sort_values(['simlarity'],ascending=False).index[1]].strip()

'“Th rough selfl  ess service, you will always \nbe fruitful and fi  nd the fulfi  llment of your \ndesires”  this is the promise of the Creator'

In [44]:
## used for correct the spell mistake
import textblob
obj = textblob.TextBlob(clean_text[df.sort_values(['simlarity'],ascending=False).index[0]].strip())
print(obj.correct().string)

Oh  rough constant off  or over many lifetime, 
a person becomes purify  ed of all self  s desires 
and attains the supreme goal of life


### semantic search in text document

In [ ]:
QAmodel = SentenceTransformer('clips/mfaq')
## to get the exact answer 

In [46]:
# Sent
util.semantic_search(embeddings[1419],embed_question)

[[{'corpus_id': 0, 'score': 0.5400464534759521}]]

In [122]:
# embeddings[0:3]

In [123]:
# util.semantic_search(embeddings[],mbed_question)

In [124]:
# embeddings

In [49]:
import pickle

## Question Answering with semantic search

In [128]:
from transformers import pipeline

In [135]:
qamodel = pipeline('question-answering')
question = """Th rough constant effort over many lifetimes"""
context = "Th  rough constant eff  ort over many lifetimes   a person becomes purifi  ed of all selfi  sh desires  and attains the supreme goal of life"
qamodel(question = question,context = context)       #  <---- it will return answer with context slicing

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'score': 0.05454063042998314,
 'start': 50,
 'end': 77,
 'answer': 'a person becomes purifi  ed'}

In [12]:
paragraphs = raw_text.split('.')

In [13]:
len(paragraphs)

3021

In [17]:
for par in paragraphs:
    print(par)
    print()

╭
the bhagavad gita
╯
On this path effort never goes to waste,
and there is no failure

 Even a little effort
toward spiritual awareness will protect you
from the greatest fear

 (2:40)
╭Also in Th  is Series
╭
the dhammapada
the upanishads╭
 Th e Bhagavad  Gita
╭
Introduced & 
Translated by  
Eknath  
Easwaran
Nilgiri Press© 1985, 2007 by Th  e Blue Mountain Center of Meditation 
All rights reserved

 Printed in Canada
Second edition

 First printing May 2007isbn–13
 : 978–1–58638–019–9
isbn–10   : 1–58638–019–2
Library of Congress Control Number: 2006934966
 
Printed on recycled paperEknath Easwaran founded the Blue Mountain Center of 
Meditation in Berkeley, California, in 1961

 Th  e Center 
is a nonproﬁ  t organization chartered with carrying on 
Easwaran’s legacy and work

 Nilgiri Press, a department of the Center, publishes books on how to lead a spiritual life in the home and community

 Th  e Center also teaches 
Easwaran’s Eight Point Program for spiritual living at retreat

In [34]:
filter_list = []
for inde ,par_len in enumerate(map(len,paragraphs)):
    if par_len>60:
        filter_list.append(paragraphs[inde])
        
        

In [35]:
len(filter_list)

2447

In [26]:
filter_list

[]

In [44]:
reader = PdfReader(r"C:\Users\Ranjit Singh\Desktop\working project\bhagwadGita\gitapdf.pdf")
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [46]:
len(raw_text)

820682

In [66]:
clean_raw = re.sub('[^a-zA-Z0-9]'," ",raw_text)

In [47]:
paragraphs = raw_text.split('.')

In [68]:
clean_text = []
for i in range(len(paragraphs)):
    clean = re.sub('[^a-zA-Z0-9]'," ",paragraphs[i])
    clean_text.append(clean)

In [69]:
print(clean_text)

['with commentar y by GAURANGA VANI PUBLISHERS RUPANUGA BHA JAN ASHRAM    Email  gauragopala gmail', 'com Website  rupanu gabhajanashram', 'com Senior Editor   Sv m  Giri Assistant Edito rs   Advaita   c rya D sa   Caitanya D sa   Priy nana Dev  D s   Gaura Gop la D s a Sanskrit   Sv m   Giri English Trans lations   Sv  m  Narasi gha and Sv m  Giri Design and Layo ut   Gaura Gop  la D sa Artwork   Dh r a lalit  De v  D s   Copyright 202 2 GAURANGA VAN I PUBLISHERS All Rights Res ervedPreface Introduction The History of Bhagavad g t  Ma gal cara a CHAPTER 1 Sainya Dar ana Observing the Armies on the Battlefield CHAPTER 2 S  khya Yoga   The Yoga of Analysis CHAPTER 3 Karma Yoga   The Yoga of Action CHAPTER 4 J  na Yoga   The Yoga of Knowledge CHAPTER 5 Karma Sanny sa Yoga   The Yoga of the Renunciation of Action CHAPTER 6 Dhy na Yoga   The Yoga of Meditation CHAPTER 7 J  na Vij  na Yoga The Yoga of Knowledge and RealisationContents ix xi xv xix 1 25 71 99 133 151 177CHAPTER 8 T raka Brah

### GPT-2

In [136]:
from transformers import GPT2Tokenizer , TFGPT2LMHeadModel

In [137]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gptmodel = TFGPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [153]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gptmodel = TFGPT2LMHeadModel.from_pretrained('gpt2',pad_token_id=tokenizer.eos_token_id)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [163]:
input_id = tokenizer.encode('who is dr bhim rao ambedkar',return_tensors='tf')

In [164]:
ans = gptmodel.generate(input_id,max_length=100)

In [165]:
ans

<tf.Tensor: shape=(1, 100), dtype=int32, numpy=
array([[ 8727,   318,  1553,   275, 38400,  2179,    78,   716,  3077,
        21070,     8,   198,   198,     7,    40,   716,   275, 38400,
         2179,    78,   716,  3077, 21070,     8,   198,   198,     7,
           40,   716,   275, 38400,  2179,    78,   716,  3077, 21070,
            8,   198,   198,     7,    40,   716,   275, 38400,  2179,
           78,   716,  3077, 21070,     8,   198,   198,     7,    40,
          716,   275, 38400,  2179,    78,   716,  3077, 21070,     8,
          198,   198,     7,    40,   716,   275, 38400,  2179,    78,
          716,  3077, 21070,     8,   198,   198,     7,    40,   716,
          275, 38400,  2179,    78,   716,  3077, 21070,     8,   198,
          198,     7,    40,   716,   275, 38400,  2179,    78,   716,
         3077]])>

In [166]:
print(tokenizer.decode(ans[0],skip_special_tokens=True,clean_up_tokenization_spaces=True))

who is dr bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambedkar)

(I am bhim rao ambed
